# Video 1: pyspark introduction and installation

In [ ]:
# !pip install pyspark

In [2]:
import pyspark

In [3]:
import pandas as pd

In [4]:
pd.read_csv('Book1.csv')

,Name,Age,Experience
0,A,24,4
1,B,26,6
2,C,31,11


In [14]:
type(pd.read_csv('Book1.csv'))

pandas.core.frame.DataFrame

In [5]:
# Building a pyspark session

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Practice').getOrCreate()

In [7]:
spark

In [8]:
df_pyspark = spark.read.csv('Book1.csv')

In [10]:
df_pyspark.show() # header not recognized

+----+---+----------+
| _c0|_c1|       _c2|
+----+---+----------+
|Name|Age|Experience|
|   A| 24|         4|
|   B| 26|         6|
|   C| 31|        11|
+----+---+----------+



In [20]:
df_pyspark = spark.read.option('header', 'true').csv('Book1.csv')
# Header issue fixed

In [21]:
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [23]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Experience: string (nullable = true)



# Video 2: pyspark dataframes

In [52]:
# pyspark df
# read dataset
# check datatypes of columns
# columns select and indexing
# describe
# add columns
# drop columns
# Renaming columns

In [26]:
# Initiate the session if not done

In [27]:
# spark.read.options('header', 'true').csv('Book1.csv')

In [28]:
## Check the schema (datatypes)
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Experience: string (nullable = true)



In [29]:
# By default considers all variables as string

In [30]:
df_pyspark = spark.read.option('header', 'true').csv('Book1.csv', inferSchema=True)

In [31]:
df_pyspark.show()

+----+---+----------+
|Name|Age|Experience|
+----+---+----------+
|   A| 24|         4|
|   B| 26|         6|
|   C| 31|        11|
+----+---+----------+



In [32]:
## Check the schema (datatypes)
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)



In [33]:
# Another way
# df_pyspark = spark.read.csv('Book1.csv', header=True, inferSchema=True)

In [34]:
df_pyspark.columns # Like python

['Name', 'Age', 'Experience']

In [36]:
df_pyspark.head(3) # It is row wise not like pandas

[Row(Name='A', Age=24, Experience=4),
 Row(Name='B', Age=26, Experience=6),
 Row(Name='C', Age=31, Experience=11)]

In [37]:
df_pyspark.select('Name').show()

+----+
|Name|
+----+
|   A|
|   B|
|   C|
+----+



In [38]:
type(df_pyspark.select('Name'))

pyspark.sql.dataframe.DataFrame

In [39]:
df_pyspark.select(['Name', 'Experience']).show() 

+----+----------+
|Name|Experience|
+----+----------+
|   A|         4|
|   B|         6|
|   C|        11|
+----+----------+



In [42]:
# df_pyspark['Name'].show() #This doesnt work on pyspark

In [43]:
df_pyspark.dtypes # llr to pandas

[('Name', 'string'), ('Age', 'int'), ('Experience', 'int')]

In [45]:
df_pyspark.describe().show()
# It takes all the column types even strings

+-------+----+-----------------+-----------------+
|summary|Name|              Age|       Experience|
+-------+----+-----------------+-----------------+
|  count|   3|                3|                3|
|   mean|null|             27.0|              7.0|
| stddev|null|3.605551275463989|3.605551275463989|
|    min|   A|               24|                4|
|    max|   C|               31|               11|
+-------+----+-----------------+-----------------+



In [48]:
### Adding columns in df

df_pyspark = df_pyspark.withColumn('Experience after 2 years', df_pyspark['Experience']+2)
df_pyspark.show()

+----+---+----------+------------------------+
|Name|Age|Experience|Experience after 2 years|
+----+---+----------+------------------------+
|   A| 24|         4|                       6|
|   B| 26|         6|                       8|
|   C| 31|        11|                      13|
+----+---+----------+------------------------+



In [50]:
##  Drop the columns

df_pyspark = df_pyspark.drop('Experience after 2 years') # can also take the list of columns
df_pyspark.show()

+----+---+----------+
|Name|Age|Experience|
+----+---+----------+
|   A| 24|         4|
|   B| 26|         6|
|   C| 31|        11|
+----+---+----------+



In [51]:
## Rename the columns

df_pyspark=df_pyspark.withColumnRenamed('Name', 'NewName')
df_pyspark.show()

+-------+---+----------+
|NewName|Age|Experience|
+-------+---+----------+
|      A| 24|         4|
|      B| 26|         6|
|      C| 31|        11|
+-------+---+----------+



# Video 3: Handling missing values

In [53]:
# Dropping columns
# Dropping rows
# Various parameters in dropping functionalities
# Handing missing values by mean, median, mode

In [59]:
sdf = spark.read.csv('book1.csv', inferSchema=True, header=True)
sdf.show(5, truncate=True)

+----+----+----------+------+
|Name| Age|Experience|Salary|
+----+----+----------+------+
|   A|  24|         4| 10000|
|   B|  26|         6| 45000|
|   C|  31|        11| 36000|
|   D|  41|         7| 42000|
|   E|null|      null| 23000|
+----+----+----------+------+
only showing top 5 rows



In [60]:
## drop columns

sdf.drop('Name').show()

+----+----------+------+
| Age|Experience|Salary|
+----+----------+------+
|  24|         4| 10000|
|  26|         6| 45000|
|  31|        11| 36000|
|  41|         7| 42000|
|null|      null| 23000|
|  34|        12| 36000|
+----+----------+------+



In [61]:
## drop NaN values

sdf.na.drop().show() # Whereever null values, all rows will get removed

+----+---+----------+------+
|Name|Age|Experience|Salary|
+----+---+----------+------+
|   A| 24|         4| 10000|
|   B| 26|         6| 45000|
|   C| 31|        11| 36000|
|   D| 41|         7| 42000|
|   F| 34|        12| 36000|
+----+---+----------+------+



In [69]:
# Drop has 3 params. how, thresh, subset

In [63]:
### how==any

# sdf.na.drop(how='all').show() #only all the values are NULL it will get drop
sdf.na.drop(how='any').show() #the above o[p] where one of null values, it gets dropped

+----+---+----------+------+
|Name|Age|Experience|Salary|
+----+---+----------+------+
|   A| 24|         4| 10000|
|   B| 26|         6| 45000|
|   C| 31|        11| 36000|
|   D| 41|         7| 42000|
|   F| 34|        12| 36000|
+----+---+----------+------+



In [65]:
sdf.na.drop(how='any', thresh=2).show() # Rows with atleast 2 non null values, it will preserve

+----+----+----------+------+
|Name| Age|Experience|Salary|
+----+----+----------+------+
|   A|  24|         4| 10000|
|   B|  26|         6| 45000|
|   C|  31|        11| 36000|
|   D|  41|         7| 42000|
|   E|null|      null| 23000|
|   F|  34|        12| 36000|
+----+----+----------+------+



In [67]:
sdf.na.drop(how='any', thresh=3).show() # Rows with atleast 3 non null values, it will preserve

+----+---+----------+------+
|Name|Age|Experience|Salary|
+----+---+----------+------+
|   A| 24|         4| 10000|
|   B| 26|         6| 45000|
|   C| 31|        11| 36000|
|   D| 41|         7| 42000|
|   F| 34|        12| 36000|
+----+---+----------+------+



In [68]:
# E got dropped since it doesnt have atleast 3 non null values

In [70]:
sdf.na.drop(how='any', subset='Experience').show() 
# Drops a record only when experience has a null value
# can also mention multiple columns

+----+---+----------+------+
|Name|Age|Experience|Salary|
+----+---+----------+------+
|   A| 24|         4| 10000|
|   B| 26|         6| 45000|
|   C| 31|        11| 36000|
|   D| 41|         7| 42000|
|   F| 34|        12| 36000|
+----+---+----------+------+



In [71]:
## filling missing values

In [76]:
sdf.fillna(99, ['Experience', 'Age']).show() 
#NA values imputed in 2 columns

+----+---+----------+------+
|Name|Age|Experience|Salary|
+----+---+----------+------+
|   A| 24|         4| 10000|
|   B| 26|         6| 45000|
|   C| 31|        11| 36000|
|   D| 41|         7| 42000|
|   E| 99|        99| 23000|
|   F| 34|        12| 36000|
+----+---+----------+------+



In [77]:
sdf.show()

+----+----+----------+------+
|Name| Age|Experience|Salary|
+----+----+----------+------+
|   A|  24|         4| 10000|
|   B|  26|         6| 45000|
|   C|  31|        11| 36000|
|   D|  41|         7| 42000|
|   E|null|      null| 23000|
|   F|  34|        12| 36000|
+----+----+----------+------+



In [82]:
## Imputer function
from pyspark.ml.feature import Imputer

imputer = Imputer(
            inputCols=['Age', 'Experience'],
            outputCols=['{}_imputed'.format(c) for c in ['Age', 'Experience']]
            ).setStrategy("mean") # can also do median or mode

In [83]:
imputer.fit(sdf).transform(sdf).show()

+----+----+----------+------+-----------+------------------+
|Name| Age|Experience|Salary|Age_imputed|Experience_imputed|
+----+----+----------+------+-----------+------------------+
|   A|  24|         4| 10000|         24|                 4|
|   B|  26|         6| 45000|         26|                 6|
|   C|  31|        11| 36000|         31|                11|
|   D|  41|         7| 42000|         41|                 7|
|   E|null|      null| 23000|         31|                 8|
|   F|  34|        12| 36000|         34|                12|
+----+----+----------+------+-----------+------------------+



# Video 4: Pyspark dataframes

In [84]:
# Filter operation
# &, |, ==
# ~

In [86]:
sdf1 = sdf.na.drop()

In [87]:
sdf1.show()

+----+---+----------+------+
|Name|Age|Experience|Salary|
+----+---+----------+------+
|   A| 24|         4| 10000|
|   B| 26|         6| 45000|
|   C| 31|        11| 36000|
|   D| 41|         7| 42000|
|   F| 34|        12| 36000|
+----+---+----------+------+



In [88]:
# Filter operations

In [91]:
# Salary of the people less than = 20k

sdf1.filter("Salary<=36000").show()

+----+---+----------+------+
|Name|Age|Experience|Salary|
+----+---+----------+------+
|   A| 24|         4| 10000|
|   C| 31|        11| 36000|
|   F| 34|        12| 36000|
+----+---+----------+------+



In [92]:
sdf1.filter("Salary<=36000").select(['Name', 'Age']).show()

+----+---+
|Name|Age|
+----+---+
|   A| 24|
|   C| 31|
|   F| 34|
+----+---+



In [93]:
# Multiple conditions

In [95]:
sdf1.filter(sdf1['Salary']<=36000).show()

+----+---+----------+------+
|Name|Age|Experience|Salary|
+----+---+----------+------+
|   A| 24|         4| 10000|
|   C| 31|        11| 36000|
|   F| 34|        12| 36000|
+----+---+----------+------+



In [98]:
sdf1.filter((sdf1['Salary']<=36000) & 
            (sdf1['Salary'] >=15000)).show() # AND operation

+----+---+----------+------+
|Name|Age|Experience|Salary|
+----+---+----------+------+
|   C| 31|        11| 36000|
|   F| 34|        12| 36000|
+----+---+----------+------+



In [99]:
sdf1.filter((sdf1['Salary']<=36000) | 
            (sdf1['Salary'] >=15000)).show() # OR operation

+----+---+----------+------+
|Name|Age|Experience|Salary|
+----+---+----------+------+
|   A| 24|         4| 10000|
|   B| 26|         6| 45000|
|   C| 31|        11| 36000|
|   D| 41|         7| 42000|
|   F| 34|        12| 36000|
+----+---+----------+------+



In [102]:
sdf1.filter(~(sdf1['Salary']<=36000)).show() # inverse filter operation

+----+---+----------+------+
|Name|Age|Experience|Salary|
+----+---+----------+------+
|   B| 26|         6| 45000|
|   D| 41|         7| 42000|
+----+---+----------+------+



# 5. Group by and Agg functions

In [103]:
# Modified the book1.csv for this module

In [104]:
# Importing the data once again

sdf = spark.read.csv('book1.csv', inferSchema=True, header=True)
sdf.show(5, truncate=True)

+----+---+----------+------+-----------+
|Name|Age|Experience|Salary| Department|
+----+---+----------+------+-----------+
|   A| 24|         4| 10000|        IOS|
|   B| 26|         6| 45000|DataScience|
|   C| 31|        11| 36000|DataScience|
|   D| 41|         7| 42000|        IOS|
|   E| 34|        12| 36000|DataScience|
+----+---+----------+------+-----------+



In [105]:
sdf.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Department: string (nullable = true)



In [112]:
## Groupby

sdf.groupBy('Department').sum('Salary').show()

+-----------+-----------+
| Department|sum(Salary)|
+-----------+-----------+
|        IOS|      52000|
|DataScience|     117000|
+-----------+-----------+



In [113]:
## Department max salary

sdf.groupBy('Department').sum('Salary').show()

+-----------+-----------+
| Department|sum(Salary)|
+-----------+-----------+
|        IOS|      52000|
|DataScience|     117000|
+-----------+-----------+



In [114]:
## Department mean salary

sdf.groupBy('Department').mean('Salary').show()

+-----------+-----------+
| Department|avg(Salary)|
+-----------+-----------+
|        IOS|    26000.0|
|DataScience|    39000.0|
+-----------+-----------+



In [115]:
sdf.groupBy('Department').count().show()

+-----------+-----+
| Department|count|
+-----------+-----+
|        IOS|    2|
|DataScience|    3|
+-----------+-----+



In [116]:
sdf.agg({'Salary': 'sum'}).show()

+-----------+
|sum(Salary)|
+-----------+
|     169000|
+-----------+



In [117]:
sdf.groupBy('Department').max('Salary').show()

+-----------+-----------+
| Department|max(Salary)|
+-----------+-----------+
|        IOS|      42000|
|DataScience|      45000|
+-----------+-----------+



In [118]:
sdf.groupBy('Department').min('Salary').show()

+-----------+-----------+
| Department|min(Salary)|
+-----------+-----------+
|        IOS|      10000|
|DataScience|      36000|
+-----------+-----------+

